In [460]:
data_path=r"tam.txt"
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense 
from tensorflow.keras.models import Model

In [461]:
samples=1000

In [462]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')

    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [463]:
input_characters = sorted(list(input_characters)) 

In [464]:
target_characters = sorted(list(target_characters))

In [465]:
t=target_texts
target_texts=input_texts
input_texts=t

In [466]:
len(input_texts)

201

In [467]:
len(target_texts)

201

In [468]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [469]:
tokenizer_inputs = Tokenizer(filters='') 
tokenizer_outputs = Tokenizer(filters='')
tokenizer_inputs.fit_on_texts(input_texts) 
tokenizer_outputs.fit_on_texts(target_texts)
input_texts = tokenizer_inputs.texts_to_sequences(input_texts)
output_sequences = tokenizer_outputs.texts_to_sequences(target_texts)

In [470]:
# Vocabulary size
input_vocab_size = len(tokenizer_inputs.word_index) 
output_vocab_size = len(tokenizer_outputs.word_index) 

# Padding sequences
max_input_length = max(len(seq) for seq in input_texts) + 1
max_output_length = max(len(seq) for seq in output_sequences) + 1 

padded_input_sequences = pad_sequences(input_texts, maxlen=20, padding='post')
padded_output_sequences = pad_sequences(output_sequences, maxlen=20, padding='post')

In [471]:
max_input_length

12

In [472]:
padded_input_sequences.shape

(201, 20)

In [473]:
max_output_length

20

In [474]:
input_vocab_size 

598

In [475]:
output_vocab_size

431

In [476]:
padded_output_sequences.shape

(201, 20)

In [477]:
embedding_dim=20

In [478]:
latent_dim=128

In [479]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Encoder
encoder_inputs = Input(shape=(20,))
encoder_embedding = Embedding(input_vocab_size + 1, embedding_dim)(encoder_inputs)  # Add 1 to vocab size
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_output_length,))
decoder_embedding = Embedding(output_vocab_size + 1, embedding_dim)(decoder_inputs)  # Add 1 to vocab size
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size + 1, activation='softmax')  # Add 1 to vocab size
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [480]:
from tensorflow.keras.optimizers.legacy import Adam

model.compile(optimizer= Adam(), loss='sparse_categorical_crossentropy')

In [481]:
padded_input_sequences.shape 

(201, 20)

In [482]:
# Train the model
model.fit([padded_input_sequences, padded_output_sequences], padded_output_sequences, epochs=100)

Epoch 1/100


7/7 [==============================] - 3s 20ms/step - loss: 6.0372
Epoch 2/100
7/7 [==============================] - 0s 19ms/step - loss: 5.5933
Epoch 3/100
7/7 [==============================] - 0s 18ms/step - loss: 3.0919
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 1.8789
Epoch 5/100
7/7 [==============================] - 0s 17ms/step - loss: 1.6296
Epoch 6/100
7/7 [==============================] - 0s 18ms/step - loss: 1.5800
Epoch 7/100
7/7 [==============================] - 0s 17ms/step - loss: 1.5486
Epoch 8/100
7/7 [==============================] - 0s 17ms/step - loss: 1.5269
Epoch 9/100
7/7 [==============================] - 0s 19ms/step - loss: 1.5135
Epoch 10/100
7/7 [==============================] - 0s 18ms/step - loss: 1.5021
Epoch 11/100
7/7 [==============================] - 0s 17ms/step - loss: 1.4917
Epoch 12/100
7/7 [==============================] - 0s 18ms/step - loss: 1.4822
Epoch 13/100
7/7 [==============================] - 0s 17ms/s

In [483]:
x_test=["நான் மிகவும் சந்த ாஷமாக இருக்கிதேன","அது அவசியமில்லை"," யவுசசய்து அல மீண்டும் சசய்யவும","அது ஒரு நல்ை தயாசலை","அவர்கள் ஒன்ோக தவலை சசய்ய ஒப்புக்சகாண்டைர"]

In [484]:
y_test=["Im so happy","It wasnt necessary","Please do that again","That is good idea","They agreed to work together"]

In [492]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize test input and output texts
test_input_sequences = tokenizer_inputs.texts_to_sequences(x_test)
test_output_sequences = tokenizer_outputs.texts_to_sequences(y_test)

# Padding test sequences
padded_test_input_sequences = pad_sequences(test_input_sequences, maxlen=20, padding='post')
padded_test_output_sequences = pad_sequences(test_output_sequences, maxlen=20, padding='post')


In [502]:
padded_test_output_sequences.shape 

(5, 20)

In [504]:
model([padded_test_input_sequences ,padded_test_input_sequences])

<tf.Tensor: shape=(5, 20, 432), dtype=float32, numpy=
array([[[7.18859141e-04, 1.72264234e-03, 4.21835482e-03, ...,
         6.12144220e-07, 3.92523407e-06, 7.59035970e-07],
        [3.99596378e-04, 1.63226649e-02, 2.14929450e-02, ...,
         3.71074589e-06, 1.10515939e-05, 2.76526839e-06],
        [1.36367627e-03, 1.34672463e-01, 1.13332592e-01, ...,
         6.48132618e-06, 1.25808047e-05, 4.27751866e-06],
        ...,
        [9.98113990e-01, 1.08783424e-04, 1.98772468e-05, ...,
         9.15788405e-06, 6.19630964e-06, 1.90259525e-05],
        [9.98113990e-01, 1.08777298e-04, 1.98764319e-05, ...,
         9.15775308e-06, 6.19621505e-06, 1.90257706e-05],
        [9.98113990e-01, 1.08773667e-04, 1.98759608e-05, ...,
         9.15767396e-06, 6.19615003e-06, 1.90256615e-05]],

       [[7.70295039e-04, 6.90793153e-04, 2.10792175e-03, ...,
         7.22310631e-07, 6.48884770e-06, 1.15483044e-06],
        [3.72880808e-04, 9.69298091e-03, 1.47818560e-02, ...,
         2.80859877e-06, 1.35